# Mastering with ROS: TIAGo

<img src="img/pal-logo.png" width="400" />

<img src="img/tiago.png" width="400"/>

<img src="img/robotignite_logo_text.png" width="400"/>

## Unit 3: Motion Planning with MoveIt! Part 1

<p style="background:green;color:white;">SUMMARY</p>

Estimated time of completion: **1h**

In the following Unit you are going to check different way of performing Motion Planning with TIAGo using MoveIt:

* Planning in joint space
* Planning in cartesian space

<p style="background:green;color:white;">END OF SUMMARY</p>

## Planning in joint space

In this first section of the chapter,you are going to see how to use MoveIt! in order to bring the torso-arm group of joints of TIAGo to a desired joint space configuration, ensuring joint limit avoidance and self-collision avoidance.

In the following exercise, you are going to run an example that will bring TIAGO to the following joint space configuration of the torso-arm group:

In [ ]:
torso_joint: 0
arm_1_joint: 3.1
arm_2_joint: 0.2
arm_3_joint: -2.1
arm_4_joint: 2.0
arm_5_joint: 1.0
arm_6_joint:-0.8
arm_7_joint: 0

Note that if we try to set the above kinematic configuration moving individual joints we will end up probably in a self-collision. The node that will take care to find a plan, i.e. a sequence of movements, to reach such a kinematic configuration is **plan_arm_torso_fk**, included in **tiago_moveit_tutorial** package.

<p style="background:#EE9023;color:white;">Exercise 3.1</p>

a) Execute the following command in order to start the Motion Planning.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosrun tiago_moveit_tutorial plan_arm_torso_fk  0 3.1 0.2 -2.1 2.0 1.0 -0.8 0

<img src="img/tiago_moveit1.gif" width="600" />

<p style="background:#EE9023;color:white;">End of Exercise 3.1</p>

Great! So now, you have already created a full map of the environment. So now what? Well, now it's time to save this map, so you can use it in the Pat Planning system!

### Inspecting the code

The code to implement such a node able to plan in joint space is shown below:

**plan_arm_torso_fk.cpp**

In [ ]:
// ROS headers
#include <ros/ros.h>

// MoveIt! headers
#include <moveit/move_group_interface/move_group.h>

// Std C++ headers
#include <string>
#include <vector>
#include <map>

int main(int argc, char** argv)
{
  ros::init(argc, argv, "plan_arm_torso_fk");

  if ( argc < 9 )
  {
    ROS_INFO(" ");
    ROS_INFO("\tUsage:");
    ROS_INFO(" ");
    ROS_INFO("\trosrun tiago_moveit_tutorial plan_arm_torso_fk torso_lift arm_1 arm_2 arm_3 arm_4 arm_5 arm_6 arm_7");
    ROS_INFO(" ");
    ROS_INFO("\twhere the list of arguments are the target values for the given joints");
    ROS_INFO(" ");
    return EXIT_FAILURE;
  }

  std::map<std::string, double> target_position;

  target_position["torso_lift_joint"] = atof(argv[1]);
  target_position["arm_1_joint"] = atof(argv[2]);
  target_position["arm_2_joint"] = atof(argv[3]);
  target_position["arm_3_joint"] = atof(argv[4]);
  target_position["arm_4_joint"] = atof(argv[5]);
  target_position["arm_5_joint"] = atof(argv[6]);
  target_position["arm_6_joint"] = atof(argv[7]);
  target_position["arm_7_joint"] = atof(argv[8]);

  ros::NodeHandle nh;
  ros::AsyncSpinner spinner(1);
  spinner.start();

  std::vector<std::string> torso_arm_joint_names;
  //select group of joints
  moveit::planning_interface::MoveGroup group_arm_torso("arm_torso");
  //choose your preferred planner
  group_arm_torso.setPlannerId("SBLkConfigDefault");

  torso_arm_joint_names = group_arm_torso.getJoints();

  group_arm_torso.setStartStateToCurrentState();
  group_arm_torso.setMaxVelocityScalingFactor(1.0);

  for (unsigned int i = 0; i < torso_arm_joint_names.size(); ++i)
    if ( target_position.count(torso_arm_joint_names[i]) > 0 )
    {
      ROS_INFO_STREAM("\t" << torso_arm_joint_names[i] << " goal position: " << target_position[torso_arm_joint_names[i]]);
      group_arm_torso.setJointValueTarget(torso_arm_joint_names[i], target_position[torso_arm_joint_names[i]]);
    }

  moveit::planning_interface::MoveGroup::Plan my_plan;
  group_arm_torso.setPlanningTime(5.0);
  bool success = group_arm_torso.plan(my_plan);

  if ( !success )
    throw std::runtime_error("No plan found");

  ROS_INFO_STREAM("Plan found in " << my_plan.planning_time_ << " seconds");

  // Execute the plan
  ros::Time start = ros::Time::now();

  group_arm_torso.move();

  ROS_INFO_STREAM("Motion duration: " << (ros::Time::now() - start).toSec());

  spinner.stop();

  return EXIT_SUCCESS;
}

Basically, the key parts of the code are:

* Choose a group of joints
* Choose a planner
* Set initial and desired joint state
* Give time to find a plan
* Execute the plan if found

Note that when a plan is found and is executed with the following line of code

In [ ]:
group_arm_torso.move();

The required control commands are sent to the arm and torso controllers through their Action interfaces:

In [ ]:
/arm_controller/follow_joint_trajectory/goal
/torso_controller/follow_joint_trajectory/goal

## Planning in cartesian space

In this section, you are going to see how to use MoveIt! in order to move the end-effector frame of TIAGo to a desired pose in cartesian space. An example is given in C++.

In the following exercise, you are going to run an example that will bring TIAGo's end-effector frame, i.e. arm_tool_link, to the following cartesian space configuration with respect to /base_footprint:

In [ ]:
x: 0.4
y: -0.3
z: 0.26
Roll: -0.011
Pitch: 1.57
Yaw: 0.037

which corresponds to the following pose as shown in rviz:

<img src="img/tiago_goal_pose_planning_ik.jpg" width="400" />

In order to safely reach such a cartesian goal, you will use the node **plan_arm_torso_ik**, included in **tiago_moveit_tutorial** package.

<p style="background:#EE9023;color:white;">**Exercise 3.2**</p>

a) Execute the following command in order to start the Motion Planning.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #3</p>
</th>
</tr>
</table>

In [ ]:
rosrun tiago_moveit_tutorial plan_arm_torso_ik 0.4 -0.3 0.26 -0.011, 1.57, 0.037

You should see something like this:

<img src="img/tiago_moveit2.gif" width="600" />

Note that the final pose of /arm_tool_link is the desired one:

<img src="img/mani_links.png" width="600" />

<p style="background:#EE9023;color:white;">**End of Exercise 3.2**</p>

### Inspecting the code

The code to implement such a node able to plan in cartesian space is shown below:

**plan_arm_torso_ik.cpp**

In [ ]:
// ROS headers
#include <ros/ros.h>

// MoveIt! headers
#include <moveit/move_group_interface/move_group.h>

// Std C++ headers
#include <string>
#include <vector>
#include <map>

int main(int argc, char** argv)
{
  ros::init(argc, argv, "plan_arm_torso_ik");

  if ( argc < 7 )
  {
    ROS_INFO(" ");
    ROS_INFO("\tUsage:");
    ROS_INFO(" ");
    ROS_INFO("\trosrun tiago_moveit_tutorial plan_arm_torso_ik  x y z  r p y");
    ROS_INFO(" ");
    ROS_INFO("\twhere the list of arguments specify the target pose of /arm_tool_link expressed in /base_footprint");
    ROS_INFO(" ");
    return EXIT_FAILURE;
  }

  geometry_msgs::PoseStamped goal_pose;
  goal_pose.header.frame_id = "base_footprint";
  goal_pose.pose.position.x = atof(argv[1]);
  goal_pose.pose.position.y = atof(argv[2]);
  goal_pose.pose.position.z = atof(argv[3]);
  goal_pose.pose.orientation = tf::createQuaternionMsgFromRollPitchYaw(atof(argv[4]), atof(argv[5]), atof(argv[6]));

  ros::NodeHandle nh;
  ros::AsyncSpinner spinner(1);
  spinner.start();

  std::vector<std::string> torso_arm_joint_names;
  //select group of joints
  moveit::planning_interface::MoveGroup group_arm_torso("arm_torso");
  //choose your preferred planner
  group_arm_torso.setPlannerId("SBLkConfigDefault");
  group_arm_torso.setPoseReferenceFrame("base_footprint");
  group_arm_torso.setPoseTarget(goal_pose);

  ROS_INFO_STREAM("Planning to move " <<
                  group_arm_torso.getEndEffectorLink() << " to a target pose expressed in " <<
                  group_arm_torso.getPlanningFrame());

  group_arm_torso.setStartStateToCurrentState();
  group_arm_torso.setMaxVelocityScalingFactor(1.0);


  moveit::planning_interface::MoveGroup::Plan my_plan;
  //set maximum time to find a plan
  group_arm_torso.setPlanningTime(5.0);
  bool success = group_arm_torso.plan(my_plan);

  if ( !success )
    throw std::runtime_error("No plan found");

  ROS_INFO_STREAM("Plan found in " << my_plan.planning_time_ << " seconds");

  // Execute the plan
  ros::Time start = ros::Time::now();

  group_arm_torso.move();

  ROS_INFO_STREAM("Motion duration: " << (ros::Time::now() - start).toSec());

  spinner.stop();

  return EXIT_SUCCESS;
}

Note that the key parts of the code are:

* Choose a group of joints
* Choose a planner and define the reference frame, i.e. /base_footprint in this case
* Set desired pose of /arm_tool_link encoded in a geometry_msgs::PoseStamped
* Give time to find a plan
* Execute the plan if found

Note that when a plan is found and is executed with the following line of code

In [ ]:
group_arm_torso.move();

The required control commands are sent to the arm and torso controllers through their Action interfaces:

In [ ]:
/arm_controller/follow_joint_trajectory/goal
/torso_controller/follow_joint_trajectory/goal